## B-cell algorithm applied to NES problem as inspired by Andrej Karpathy 

I came across an implementation of Natural Evolution Strategies (NES), and thought I would use a variant of the B-cell algorithm (BCA) to see how the algorithm performs on the same problem.

This notebook was inspired by Jovan Sardinha's excellent repo:

https://github.com/jovsa/evolution-strategies-exploration/

which was in turn inspired by the one-&-only Andrej Karpathy's code found at 

https://gist.github.com/karpathy/77fbb6a8dac5395f1b73e7a89300318d

The general idea of the original code, by Karpathy and Sardinha, is to apply evolutionary strategies in the context of reinforcement learning. Further details can be found at:

https://arxiv.org/abs/1703.03864
https://medium.com/beyond-intelligence/reinforcement-learning-or-evolutionary-strategies-nature-has-a-solution-both-8bc80db539b3

It seemed likely that the other nature-inspired algorithms could also be used in this problem domain, so I designed a variant of the BCA which could be used on a GPU (for pretty obvious perfomance reasons!).

This is the original BCA paper:

Kelsey J, Timmis J. Immune inspired somatic contiguous hypermutation for function optimisation Genetic and Evolutionary Computation (GECCO 2003); 2003. July 12–16; Chicago, USA: Springer; 2003: 207–218

There are a number of other papers on the BCA, & related nature-inspired algorithms, which you can google if you're interested.

I willl  desribe the various aspects of the BCA in the code below. 

In [4]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"{device}")

print("Success is not final. Failure is not final. It is the courage to continue that counts")

cuda:0
Success is not final. Failure is not final. It is the courage to continue that counts


## How the B-cell algorithm works

The BCA is a nature-inspired algorithm based on the way in which the immune system creates antibodies to invading organisms. It creates an initially random small repertoire of "antibodies" - candidate solutions to a particular function. It then evaluates them, clones them, and  subjects them to hypermutation - modifying them by adding or subtracting random vectors. The clones are then evaluated: if they have a higher affinity than the original antibody, they replace it in the repertoire. This process loops until the maximum number of iterations, or until a threshold distance has been reached - i.e. the algorithm has converged on an antibody that is sufficiently close to the required value.

The BCA usually acts on the bit level, but I couldn't find a way to implement this in `PyTorch` without doing a deep dive on the mysteries of operators like `torch.logical_xor()`. However, it is relatively simple to implement adding a small random vector to an "antibody", which is the method used below. It would be interesting to see how the typical BCA mutation operator works, and this is work I might implement at a later date - this is intended simply as a demonstration.  

The functions for adapting the initial repertoire of antibodies is given below, followed by a description.

In [2]:
def clone_repertoire1d(antibodies, number_of_clones, device='cuda:0'):
    """Takes a vector of receptors/antibodies, clones them, returns the clones."""
    clones = torch.zeros((antibodies.shape[0], number_of_clones, antibodies.shape[1])).to(device)
    for idx in range(antibodies.shape[0]):
        clones[idx] = clone_single_antibody(antibody=antibodies[idx], number_of_clones=number_of_clones, device=device)
    return clones

def mutate_repertoire1d(clones, eps=2.5, device='cuda:0'):
    """Takes a 2D tensor of shape (number_of_clones, vector_size) & mutates them, returns the mutated clones."""
    for idx in range(clones.shape[0]):
        clones[idx] = mutate(clones=clones[idx], eps=eps, device=device)
    return clones

def clone_single_antibody(antibody, number_of_clones, device='cuda:0'):
    """Produces specified number of clones of the given antibody."""
    return antibody.repeat(number_of_clones, 1).to(device)
        
def get_mutations(M, N, epsilon, device='cuda:0'):
    """Returns a 2D tensor sampled from a Normal distribution with a standard deviaiton given by epsilon."""
    return torch.empty(size=(M, N,)).normal_(mean=0., std=epsilon).to(device)

def mutate(clones, eps, device='cuda:0'):
    """Adds a random perturbation to the given clones."""
    mutations = get_mutations(M=clones.shape[0], N=clones.shape[1], epsilon=eps, device=device)
    return clones + mutations

def get_dev_rate(update_step_, epsilon_0=2.5, decay_=0.1):
    """An exponential decay for the epsilon parameter."""
    update_step, decay = torch.tensor(update_step_), torch.tensor(decay_)
    return epsilon_0 * torch.exp(-(update_step * decay))

### BCA: main adaptive process

The BCA, as noted, ordinarily operates at the bit-level representation of floating-point numbers, as given by the IEEE-754 standard. It has been applied to other problem domains, but function optimisation in the real domain makes it easy to assess it's relative performance against other, related algorithms. Here, we use vectors drawn from a normal distribution with a standard deviation given by the $\epsilon$ parameter. 

The BCA seeks to optimise functions inspired by how the immune system creates antibodies, which are receptors on immune system cells. The cells are exposed to fragments of a target, which could be viral or bacterial in nature. These fragments are known collectively as *antigen*. The cells initially have a relatively weak affinity for/response to the antigen. They are cloned, and the clones then undergo a massive rearrangement of their genes (along with other adaptive processes) known as *somatic hypermutation*. The clones are then exposed to the antigen, and those which show greater affinity are selected to undergo the cloning and hypermutation process, until a suitably high affinity for the antigen has been achieved.  

So, how does the BCA work? The basic idea of the BCA, in general terms, is the following:

- create an initial repertoire of randomly-initialised antibodies - our candidiate solutions to the function we're trying to optimise, which are random vectors of flaoting point numbers, or in this case `torch` tensors
- evaluate the repertoire
- create a clonal pool for each of the antibodies in the repertoire
- for each clone in the clonal pool:
    - subject it to *hypermutation*: generate a (small) random vector and add it to the clone
- once each clone has been hypermutated, it is then evaluated to find its affinity
- the process continues, until we run out of iterations or we have converged to within a given distance of the solution 

This is the basic B-cell algorithm, although the hypermutation operator implemented below is rather different to the original version, which relied on applying hypermutation at the bit level.

### Modifying the $\epsilon$ parameter

The $\epsilon$ parameter governs the standard deviation of the normal distribution from which we sample our random numbers. 

Since the initial repertoire is random, we are likely (unless we get very lucky) to need to make large changes to the antibody. So our hypermutations will be correspondingly large. However, as we obtain antibodies (tensors) with higher affinity - those with a smaller distance to the *target tensor* - the rate of mutation should reduce, since we are closer to the target. 

There are a variety of ways of modifying $\epsilon$. One could implement an exponential decay, for example - some code to do just this is included above, for completeness. A number of strategies exist to reduce the learning rate of a deep neural network, whcih is an analogous problem. However, I found that a very simple method actually performed the best - simply by initially setting $\epsilon = 1.$, and then halving it every $N$ iterations. This prevents the hypermutation operator from pushing the mutated clone away from the optimal solution, by adding (or subtracting, of course) a random vector of too large a magnitude. As the antibodies develop higher affinity, the mutations reduce until it's within the specified tolerance.

It would be interesting to do further research on decreasing the $\epsilon$ parameter.

# The original NES code

This code can be found at:

https://github.com/jovsa/evolution-strategies-exploration/blob/master/notebooks/bare%20bones%20implementation%20of%20NES%20-%20karpathy.ipynb

the notebook and repository written by Jovan Sardinha. This is the `jupyter` notebook which inspired this piece of work; there's also a thorough analysis of other aspects of the NES strategy, which Jovan goes into in depth.

In turn, it's based on the following gist by Andrej Karpathy:

https://gist.github.com/karpathy/77fbb6a8dac5395f1b73e7a89300318d
 
I thought it would be useful to reproduce it here, so that comparisions between the two algorithms can be made; the code is taken from the above-cited notebook, and all credit and copyright for the cell below should be given to Jovan Sardinha for his work. I have, however, mdae one alteration for purposes of comparison - I changed the *total number of iterations*, so that the two algorithms can be directly compared.

In [19]:
# taken from: https://github.com/jovsa/evolution-strategies-exploration/blob/master/notebooks/bare%20bones%20implementation%20of%20NES%20-%20karpathy.ipynb

"""
A bare bones examples of optimizing a black-box function (f) using
Natural Evolution Strategies (NES), where the parameter distribution is a 
gaussian of fixed standard deviation.

source: https://gist.github.com/karpathy/77fbb6a8dac5395f1b73e7a89300318d
"""

import numpy as np
np.random.seed(0)

# the function we want to optimize
def f(w):
  # here we would normally:
  # ... 1) create a neural network with weights w
  # ... 2) run the neural network on the environment for some time
  # ... 3) sum up and return the total reward

  # but for the purposes of an example, lets try to minimize
  # the L2 distance to a specific solution vector. So the highest reward
  # we can achieve is 0, when the vector w is exactly equal to solution
  reward = -np.sum(np.square(solution - w))
  return reward

# hyperparameters
nrepertoire = 50 # repertoireulation size
sigma = 0.1 # noise standard deviation
alpha = 0.001 # learning rate

# start the optimization
solution = np.array([0.5, 0.1, -0.3])
w = np.random.randn(3) # our initial guess is random
for i in range(1001):

  # print current fitness of the most likely parameter setting
  if i % 20 == 0:
    print('iter %d. w: %s, solution: %s, reward: %f' % 
          (i, str(w), str(solution), f(w)))

  # initialize memory for a repertoireulation of w's, and their rewards
  N = np.random.randn(nrepertoire, 3) # samples from a normal distribution N(0,1)
  R = np.zeros(nrepertoire)
  for j in range(nrepertoire):
    w_try = w + sigma*N[j] # jitter w using gaussian of sigma 0.1
    R[j] = f(w_try) # evaluate the jittered version

  # standardize the rewards to have a gaussian distribution
  A = (R - np.mean(R)) / np.std(R)
  # perform the parameter update. The matrix multiply below
  # is just an efficient way to sum up all the rows of the noise matrix N,
  # where each row N[j] is weighted by A[j]
  w = w + alpha/(nrepertoire*sigma) * np.dot(N.T, A)

iter 0. w: [1.76405235 0.40015721 0.97873798], solution: [ 0.5  0.1 -0.3], reward: -3.323094
iter 20. w: [1.63796944 0.36987244 0.84497941], solution: [ 0.5  0.1 -0.3], reward: -2.678783
iter 40. w: [1.50042904 0.33577052 0.70329169], solution: [ 0.5  0.1 -0.3], reward: -2.063040
iter 60. w: [1.36438269 0.29247833 0.56990397], solution: [ 0.5  0.1 -0.3], reward: -1.540938
iter 80. w: [1.2257328  0.25622233 0.43607161], solution: [ 0.5  0.1 -0.3], reward: -1.092895
iter 100. w: [1.08819889 0.22827364 0.30415088], solution: [ 0.5  0.1 -0.3], reward: -0.727430
iter 120. w: [0.95675286 0.19282042 0.16682465], solution: [ 0.5  0.1 -0.3], reward: -0.435164
iter 140. w: [0.82214521 0.16161165 0.03600742], solution: [ 0.5  0.1 -0.3], reward: -0.220475
iter 160. w: [ 0.70282088  0.12935569 -0.09779598], solution: [ 0.5  0.1 -0.3], reward: -0.082885
iter 180. w: [ 0.58380424  0.11579811 -0.21083135], solution: [ 0.5  0.1 -0.3], reward: -0.015224
iter 200. w: [ 0.52089064  0.09897718 -0.2761225 ]

# B-cell algorithm applied to the NES problem

Below is the application of the B-cell algorithm variant, as applied to the same problem.

First, we set up the target tensor. We're using `torch` to speed up running a large number of trials, so it can be run on a GPU if one is available, hence moving it to the `device`.  
```
# target tensor
solution = torch.tensor([0.5, 0.1, -0.3]).to(device)
```

We then set up a tolerance. If an antibody in the repertoire is within a distance less than the given tolerance, we terminate the trial. The tolerance can, of course, be set to be arbitrarily small. 
```
tolerance = 1e-5
```
We then set up the size of the initial repertoire, the size of the clonal pool. and the update step, which is the number of iterations where we modify the $\epsilon$ parameter.
```
repertoire_size = 20
number_of_clones = 100
update_step = 20
```
We also set up oour initial value for $\epsilon$:
```
_eps = 1.0
```
We iterate over a number of trials $T$, randomly initialise the repertoire, clone the repertoire and hypermutate, then evaluate the affinity of the clones. :
```
clones = clone_repertoire1d(antibodies=repertoire, number_of_clones=number_of_clones, device=device)
# ...
clones = mutate_repertoire1d(clones, eps=eps, device=device)
affinity = torch.sum(torch.sqrt(torch.pow(repertoire - solution, 2)), dim=1)
clone_affinity = torch.sum(torch.sqrt(torch.pow(clones - solution, 2)), dim=2)
```
Oncw we have the affinity, given above, we compare it to the original antibody on which the clones were based. If a mutated clone has a higher affinity (smaller distance) to the solution, the antibody is replaced by the mutated clone.

The (very simple!) update to the $\epsilon$ parameter happens every `update_period` iterations; in the code below, every 20 iterations, we halve `eps`.  
```
if iteration % update_period == 0:
    eps /= 2.
```
We continue applying the B-cell algorithm until either

- we reach the maximum number of iterations (i.e. fail to converge on the solution)
- an antibody is within the given `tolerance` (distance) of the solution

We set up the number of iterations to be as Karpathy's original code, which is $1000$ iterations. As can be seen, the BCA tends to converge on the solution within the given number of iterations; also, the `repertoire` tends to be more accurate (closer to the solution) than the NES algorithm results, which can be seen above.



In [24]:
# target tensor
solution = torch.tensor([0.5, 0.1, -0.3]).to(device)

tolerance = 1e-5

repertoire_size = 20
number_of_clones = 50
update_step = 20

from itertools import count, takewhile
def frange(start, stop, step):
    return takewhile(lambda x: x > stop, count(start, step))

_eps = 1.0

for t in range(100S):
    eps = _eps
    repertoire = torch.randn((10, 3)) * repertoire_size
    repertoire = repertoire.to(device)

    affinity = torch.zeros((repertoire.shape[0],)).to(device)
    clone_affinity = torch.zeros((number_of_clones,)).to(device)

    for iteration in range(1001):
        clones = clone_repertoire1d(antibodies=repertoire, number_of_clones=number_of_clones, device=device)
        if iteration % update_period == 0:
            eps /= 2.
        clones = mutate_repertoire1d(clones, eps=eps, device=device)
        affinity = torch.sum(torch.sqrt(torch.pow(repertoire - solution, 2)), dim=1)
        clone_affinity = torch.sum(torch.sqrt(torch.pow(clones - solution, 2)), dim=2)
        indexes_of_smallest_distance = torch.argmin(clone_affinity, dim=1)
    
        for idx in range(indexes_of_smallest_distance.shape[0]):
            this_clone_affinity = clone_affinity[idx][indexes_of_smallest_distance[idx]]
            if this_clone_affinity < affinity[idx]:
                repertoire[idx] = clones[idx][indexes_of_smallest_distance[idx]]
                affinity[idx] = this_clone_affinity

        mean_affinity = torch.sum(affinity)
        
        if mean_affinity < tolerance:
            print(f"Terminating {t}\t{iteration}\t{mean_affinity}\t{eps}")
            break

    # we could check out whether or not this is within a tolerance, & then implement early stopping
    if iteration == 1000:
        mean_affinity = torch.sum(affinity)
        print(f"{t}\t{iteration}\t{mean_affinity}\t{eps}")

Terminating 0	854	9.797513484954834e-06	3.814697265625e-06
Terminating 1	833	9.901821613311768e-06	7.62939453125e-06
Terminating 2	853	9.931623935699463e-06	3.814697265625e-06
Terminating 3	853	9.410083293914795e-06	3.814697265625e-06
Terminating 4	841	9.469687938690186e-06	7.62939453125e-06
Terminating 5	854	9.98377799987793e-06	3.814697265625e-06
Terminating 6	852	9.618699550628662e-06	3.814697265625e-06
Terminating 7	834	9.79006290435791e-06	7.62939453125e-06
Terminating 8	856	9.94652509689331e-06	3.814697265625e-06
Terminating 9	858	9.752810001373291e-06	3.814697265625e-06
Terminating 10	845	9.879469871520996e-06	7.62939453125e-06
Terminating 11	853	9.909272193908691e-06	3.814697265625e-06
Terminating 12	859	9.648501873016357e-06	3.814697265625e-06
Terminating 13	850	9.879469871520996e-06	3.814697265625e-06
Terminating 14	839	9.544193744659424e-06	7.62939453125e-06
Terminating 15	856	9.819865226745605e-06	3.814697265625e-06
Terminating 16	850	9.894371032714844e-06	3.814697265625e-0

Here's the repertoire, after the final trial; as can be seen, this is a good approximation of the solution.

In [21]:
repertoire

tensor([[ 0.5000,  0.1000, -0.3000],
        [ 0.5000,  0.1000, -0.3000],
        [ 0.5000,  0.1000, -0.3000],
        [ 0.5000,  0.1000, -0.3000],
        [ 0.5000,  0.1000, -0.3000],
        [ 0.5000,  0.1000, -0.3000],
        [ 0.5000,  0.1000, -0.3000],
        [ 0.5000,  0.1000, -0.3000],
        [ 0.5000,  0.1000, -0.3000],
        [ 0.5000,  0.1000, -0.3000]], device='cuda:0')